## Sampleの Dockerfile をつくります

## ClowdWatchでスケジュールされたLambdaの作成

- 適当に関数を作ります（Hello World）

### 以下疑問

- 特定のLambdaへのアクセス権（CloudWatchとかS3とか）をつけるには
- CloudWatch側の情報をLambdaに設定する必要がある

- CloudWatchはデフォルトで連携してるぽい
- デザイナーを見ると
    - トリガーと、関数がアクセスできるリソースが表示してある
    - トリガーの設定（CloudWatch Event） ここから作成する？
        - EventPattern
            - AutoScaling, EC2, Health, RDS, S3
    - CloudWatchを見てみる
        - LogGroupは自動生成されている
        - Event > Ruleも生成されている
            - RuleでのEventからメッセージを送ることは可能なのだろうか

- AWS::Lambda::Permission
    - lambda:InvokeFunction を設定

## LambdaでAutoScalingを通して、インスタンスを機動する

* 適当にインスタンスを立てます
* 適当にオートスケーリングの設定をします
* Lambdaからオートスケーリングの値を書き換えます

```python
response = client.update_auto_scaling_group(
)
response = client.update_auto_scaling_group(
    AutoScalingGroupName=group_name,
    MinSize=1,
    MaxSize=1,
    DesiredCapacity=1
)
logger.info("Started Train Cluster: %s" % str(response))
```

## CloudWatchからAutoScalingのステータス変更をキャッチしてイベントをコール

- CloudWatchを経由してAutoScalingのイベントが送られてくる…?
- LambdaからTaskをスタートする

```python
if "action" in event:
    action = event["action"]
# こういう設定ができるのか…
elif event.get("source") == "aws.autoscaling" 
  and event.get("detail-type") == "EC2 Instance Launch Successful" and event.get("detail") is not None:
    detail_dict = event.get("detail")
    if detail_dict.get("AutoScalingGroupName") == group_name:
        action = "start"
```

## Lambdaからタスクを開始する

```python
response = client.run_task(
    cluster="${ClusterArn}",
    taskDefinition="${EnvironmentName}-hso-train-taskdef",
    count=1
)
logger.info("Started ECS Task: %s" % str(response))
```

## CloudWatchでESCのタスクの終了・エラーを取得する